In [2]:
import os
import shutil
import re

In [3]:
os.getcwd()

'D:\\workspace\\jupyter\\VAE-AKF\\jupy'

In [4]:
def my_filter(path):
    log_path = os.path.join('..', path, 'log.out')
    log_s = open(log_path, 'r').readlines()
    test_path = os.path.join('..', path, 'test.out')
    test_s = open(log_path, 'r').readlines()
    #if False:
        #print(log_path)
        #return False
    #elif s[4] != 'history_length: 40\n' and s[5] != 'forward_length: 160\n':
        #print(log_path)
        #return False
    #elif 'Error' in test_s[-2]:
        #print(log_path)
        #return False
    #else:
    return True
    
def generating_dir(base_dir, root_dir):
    ret = []
    for file in os.listdir(os.path.join(root_dir, base_dir)):
        if file == 'tmp':
            continue
        path = os.path.join(base_dir, file)
        if os.path.isdir(os.path.join(root_dir, path)):
            ret = ret + generating_dir(path, root_dir)
        elif file == 'log.out':                                                   #and os.path.exists(os.path.join(root_dir, base_dir, 'best.pth')):
            if my_filter(os.path.split(path)[0]):
                ret.append(os.path.split(path)[0])
            else:
                continue
    return ret

path_list = generating_dir('ckpt'+os.path.sep, '..'+os.path.sep)
print(path_list)


['ckpt\\cstr\\srnn\\srnn_\\2020-11-25_14-57-52', 'ckpt\\cstr\\srnn\\srnn_model.D=1\\2020-12-02_13-04-20', 'ckpt\\cstr\\srnn\\srnn_model.D=3\\2020-12-02_15-44-21', 'ckpt\\cstr\\srnn\\srnn_model.D=5\\2020-12-02_19-07-53', 'ckpt\\cstr\\srnn_overshooting\\srnn_model.D=10\\2020-11-30_13-36-16', 'ckpt\\cstr\\srnn_overshooting\\srnn_model.D=10\\2020-11-30_15-34-41', 'ckpt\\cstr\\srnn_overshooting\\srnn_model.D=5\\2020-11-30_13-38-58', 'ckpt\\cstr\\vaecl\\vaecl_model.D=1\\2020-11-29_14-11-52', 'ckpt\\cstr\\vaecl\\vaecl_model.D=1\\2020-12-01_07-13-00', 'ckpt\\cstr\\vaecl\\vaecl_model.D=1\\2020-12-01_07-13-54', 'ckpt\\cstr\\vaecl\\vaecl_model.D=3\\2020-12-01_07-40-36', 'ckpt\\cstr\\vaecl\\vaecl_model.D=3\\2020-12-01_07-41-34', 'ckpt\\cstr\\vaecl\\vaecl_model.D=5\\2020-11-29_17-21-47', 'ckpt\\cstr\\vaecl\\vaecl_model.D=5\\2020-12-01_08-43-09', 'ckpt\\cstr\\vaecl\\vaecl_model.D=5\\2020-12-01_09-30-04', 'ckpt\\cstr\\vaecl_net_decoder\\vaecl_model.D=1\\2020-11-30_07-00-34', 'ckpt\\cstr\\vaecl_net_de

In [5]:
#检查log文件#可不可以直接找不存在best.pth的路径
error_log_list=[]                    #错误log文件路径集合
for t in range(len(path_list)):      #第t个临时列表中的路径
    error_log=True                   #存在error的标志
    #print('t=',t)
    f = open('..'+os.path.sep+path_list[t]+os.path.sep+'log.out','r')
    log_data = f.readlines()
    f.close()
    for line in range(len(log_data)):
        if re.search('Early stopping at epoch',log_data[line]):
            error_log=False
            #print(line,len(log_data))
            #print('find it')
    if error_log==True:
        error_log_list.append(path_list[t])
        print(path_list[t])
        print('find error')

ckpt\cstr\srnn_overshooting\srnn_model.D=10\2020-11-30_13-36-16
find error
ckpt\cstr\vrnn\vrnn_model.D=5\2020-12-01_16-34-03
find error
ckpt\west\srnn\vaecl_\2020-11-25_14-19-29
find error
ckpt\west\srnn_overshooting\srnn_model.D=5\2020-11-30_13-36-19
find error
ckpt\west\vaecl_net_decoder\vaecl_model.D=20\2020-12-01_22-21-14
find error
ckpt\west\vrnn\vrnn_model.D=1\2020-12-01_16-34-06
find error
ckpt\west\vrnn\vrnn_model.D=1\2020-12-03_01-07-19
find error
ckpt\winding\srnn\vaecl_\2020-11-25_14-19-27
find error
ckpt\winding\srnn_overshooting\srnn_model.D=10\2020-11-30_13-36-18
find error
ckpt\winding\srnn_overshooting\srnn_model.D=5\2020-11-30_13-36-17
find error
ckpt\winding\vrnn\vrnn_model.D=1\2020-12-01_16-34-03
find error
ckpt\winding\vrnn\vrnn_model.D=1\2020-12-03_03-34-50
find error
ckpt\winding\vrnn\vrnn_model.D=3\2020-12-01_16-34-04
find error
ckpt\winding\vrnn\vrnn_model.D=5\2020-12-01_16-34-05
find error
ckpt\winding\vrnn\vrnn_model.D=5\2020-12-03_01-07-18
find error


In [8]:
def Delete_log(path):
    print("function have been triggered")
    print(path)
    shutil.rmtree(path)         #删除当前目录文件夹以及文件夹内的所有文件
    path=os.path.dirname(path)
    print(path)
    if not os.listdir(path):
        Delete_log(path)
    else:
        print("Delete accomplished")
        return 0
   

In [9]:
Delete_list=[]
if error_log_list:
    print("log文件报错路径如下：")
    for line in range(len(error_log_list)):
        Delete_list.append(error_log_list[line])
        print(Delete_list[line])
    ju=input("确定删除路径文件及其所在文件夹？ 请输入'yes' or 'no'")
    if ju=='yes':
        for line in range(len(Delete_list)):
            print(line)
            Delete_log('..'+os.path.sep+Delete_list[line])
    else:
        print("删除取消，保留路径文件")
else:
    print('未发现log文件报错')

log文件报错路径如下：
ckpt\cstr\srnn_overshooting\srnn_model.D=10\2020-11-30_13-36-16
ckpt\cstr\vrnn\vrnn_model.D=5\2020-12-01_16-34-03
ckpt\west\srnn\vaecl_\2020-11-25_14-19-29
ckpt\west\srnn_overshooting\srnn_model.D=5\2020-11-30_13-36-19
ckpt\west\vaecl_net_decoder\vaecl_model.D=20\2020-12-01_22-21-14
ckpt\west\vrnn\vrnn_model.D=1\2020-12-01_16-34-06
ckpt\west\vrnn\vrnn_model.D=1\2020-12-03_01-07-19
ckpt\winding\srnn\vaecl_\2020-11-25_14-19-27
ckpt\winding\srnn_overshooting\srnn_model.D=10\2020-11-30_13-36-18
ckpt\winding\srnn_overshooting\srnn_model.D=5\2020-11-30_13-36-17
ckpt\winding\vrnn\vrnn_model.D=1\2020-12-01_16-34-03
ckpt\winding\vrnn\vrnn_model.D=1\2020-12-03_03-34-50
ckpt\winding\vrnn\vrnn_model.D=3\2020-12-01_16-34-04
ckpt\winding\vrnn\vrnn_model.D=5\2020-12-01_16-34-05
ckpt\winding\vrnn\vrnn_model.D=5\2020-12-03_01-07-18


确定删除路径文件及其所在文件夹？ 请输入'yes' or 'no' yes


0
function have been triggered
..\ckpt\cstr\srnn_overshooting\srnn_model.D=10\2020-11-30_13-36-16
..\ckpt\cstr\srnn_overshooting\srnn_model.D=10
Delete accomplished
1
function have been triggered
..\ckpt\cstr\vrnn\vrnn_model.D=5\2020-12-01_16-34-03
..\ckpt\cstr\vrnn\vrnn_model.D=5
Delete accomplished
2
function have been triggered
..\ckpt\west\srnn\vaecl_\2020-11-25_14-19-29
..\ckpt\west\srnn\vaecl_
function have been triggered
..\ckpt\west\srnn\vaecl_
..\ckpt\west\srnn
Delete accomplished
3
function have been triggered
..\ckpt\west\srnn_overshooting\srnn_model.D=5\2020-11-30_13-36-19
..\ckpt\west\srnn_overshooting\srnn_model.D=5
Delete accomplished
4
function have been triggered
..\ckpt\west\vaecl_net_decoder\vaecl_model.D=20\2020-12-01_22-21-14
..\ckpt\west\vaecl_net_decoder\vaecl_model.D=20
function have been triggered
..\ckpt\west\vaecl_net_decoder\vaecl_model.D=20
..\ckpt\west\vaecl_net_decoder
Delete accomplished
5
function have been triggered
..\ckpt\west\vrnn\vrnn_model.D=1\202